In [47]:
from keras.datasets import fashion_mnist
from art.utils import load_mnist
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, Lambda, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Sequential,load_model
from art.metrics.metrics import empirical_robustness, clever_t, clever_u, clever, loss_sensitivity, wasserstein_distance
import numpy as np
from art.estimators.classification.keras import KerasClassifier
import logging

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

In [2]:
model = load_model('./fmnist_CNN.h5')
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 28, 28, 64)        1792      
_________________________________________________________________
block1_act1 (Activation)     (None, 28, 28, 64)        0         
_________________________________________________________________
block1_pool1 (MaxPooling2D)  (None, 14, 14, 64)        0         
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 14, 14, 128)       73856     
_________________________________________________________________
block1_act2 (Activation)     (None, 14, 14, 128)       0         
_________________________________________________________________
block1_pool2 (MaxPooling2D)  (None, 7, 7, 128)         0         
_________________________________________________________________
block1_conv3 (Conv2D)        (None, 7, 7, 256)       

In [19]:
# import numpy as np
# w1 = model.get_weights()[0]

In [28]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
x_train = tf.expand_dims(x_train, axis = -1)
x_train = tf.keras.backend.repeat_elements(x=x_train, rep=3, axis=-1)
x_test = tf.expand_dims(x_test, axis = -1)
x_test = tf.keras.backend.repeat_elements(x=x_test, rep=3, axis=-1)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [29]:
# Get FMNIST
nb_test = 10
x_test, y_test = x_test[:nb_test], y_test[:nb_test]

In [10]:
R_L1 = 40
R_L2 = 2
R_LI = 0.1

# Test targeted clever
res0 = clever_t(model, x_test[-1], 2, 10, 5, R_L1, norm=1, pool_factor=3)
res1 = clever_t(model, x_test[-1], 2, 10, 5, R_L2, norm=2, pool_factor=3)
res2 = clever_t(model, x_test[-1], 2, 10, 5, R_LI, norm=np.inf, pool_factor=3)
logger.info("Targeted Keras: %f %f %f", res0, res1, res2)
assertNotEqual(res0, res1)
assert_not_equal(res1, res2)
assert_not_equal(res2, res0)

NameError: name 'model' is not defined

In [31]:
# Test untargeted clever
res0 = clever_u(krc, x_test[-1], 10, 5, R_L1, norm=1, pool_factor=3, verbose=False)
res1 = clever_u(krc, x_test[-1], 10, 5, R_L2, norm=2, pool_factor=3, verbose=False)
res2 = clever_u(krc, x_test[-1], 10, 5, R_LI, norm=np.inf, pool_factor=3, verbose=False)
logger.info("Untargeted Keras: %f %f %f", res0, res1, res2)
self.assertNotEqual(res0, res1)
self.assertNotEqual(res1, res2)
self.assertNotEqual(res2, res0)

TensorShape([28, 28, 3])

In [2]:
R_L1 = 40
R_L2 = 2
R_LI = 0.1

def _create_krclassifier():
    """
    To create a simple KerasClassifier for testing.
    :return:
    """
    import tensorflow as tf

    tf_version = [int(v) for v in tf.__version__.split(".")]
    if tf_version[0] == 2 and tf_version[1] >= 3:
        tf.compat.v1.disable_eager_execution()
        from tensorflow import keras
        from tensorflow.keras.models import Sequential
        from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
    else:
        import keras
        from keras.models import Sequential
        from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

    # Create simple CNN
    model = Sequential()
    model.add(Conv2D(4, kernel_size=(5, 5), activation="relu", input_shape=(28, 28, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(10, activation="softmax"))

    model.compile(
        loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(lr=0.01), metrics=["accuracy"]
    )

    # Get the classifier
    krc = KerasClassifier(model=model, clip_values=(0, 1), use_logits=False)

    return krc

logger = logging.getLogger(__name__)

In [19]:
# Get MNIST
batch_size, nb_train, nb_test = 100, 1000, 100
(x_train, y_train), (x_test, y_test), _, _ = load_mnist()
x_train, y_train = x_train[:nb_train], y_train[:nb_train]
x_test, y_test = x_test[:nb_test], y_test[:nb_test]

# Get the classifier
krc = _create_krclassifier()
krc.fit(x_train, y_train, batch_size=batch_size, nb_epochs=1, verbose=0)

# # Test targeted clever
# res0 = clever_t(krc, x_test[9], 9, 10, 5, R_L1, norm=1, pool_factor=3)
# res1 = clever_t(krc, x_test[-1], 3, 10, 5, R_L2, norm=2, pool_factor=3)
# res2 = clever_t(krc, x_test[-1], 3, 10, 5, R_LI, norm=np.inf, pool_factor=3)
# logger.info("Targeted Keras: %f %f %f", res0, res1, res2)
# # assertNotEqual(res0, res1)
# # asserts.assert_not_equal(res1, res2)
# # asserts.assert_not_equal(res2, res0)

# # Test untargeted clever
# res0 = clever_u(krc, x_test[-1], 10, 5, R_L1, norm=1, pool_factor=3, verbose=False)
# res1 = clever_u(krc, x_test[-1], 10, 5, R_L2, norm=2, pool_factor=3, verbose=False)
# res2 = clever_u(krc, x_test[-1], 10, 5, R_LI, norm=np.inf, pool_factor=3, verbose=False)
# logger.info("Untargeted Keras: %f %f %f", res0, res1, res2)
# asserts.assert_not_equal(res0, res1)
# asserts.assert_not_equal(res1, res2)
# asserts.assert_not_equal(res2, res0)


In [20]:
tmp1 = 0
for index in range(0, nb_test):
    pred = krc.predict(np.reshape(x_test[index], [-1,28,28,1]))
    res_tmp = clever_u(krc, x_test[index], 10, 5, R_LI, norm=np.inf, pool_factor=3, verbose=False)
    tmp1 = tmp1+res_tmp
    print('ID:', index, '\n', '\t',
          'pred:', np.argmax(pred), '\t',
          'ground:', np.argmax(y_test[index]), '\n', '\t',
          'T-F:', np.argmax(pred)==np.argmax(y_test[index]), '\t'
          'clever_score:', res_tmp)

ID: 0 
 	 pred: 7 	 ground: 7 
 	 T-F: True 	clever_score: 0.003328944670751221
ID: 1 
 	 pred: 6 	 ground: 2 
 	 T-F: False 	clever_score: 0.00047126426988546147
ID: 2 
 	 pred: 1 	 ground: 1 
 	 T-F: True 	clever_score: 0.0027436670340193143
ID: 3 
 	 pred: 6 	 ground: 0 
 	 T-F: False 	clever_score: 0.027589897582549358
ID: 4 
 	 pred: 4 	 ground: 4 
 	 T-F: True 	clever_score: 0.003315034312937169
ID: 5 
 	 pred: 1 	 ground: 1 
 	 T-F: True 	clever_score: 0.0048344370389878525
ID: 6 
 	 pred: 4 	 ground: 4 
 	 T-F: True 	clever_score: 0.0008780483278103667
ID: 7 
 	 pred: 9 	 ground: 9 
 	 T-F: True 	clever_score: 4.938043934320655e-06
ID: 8 
 	 pred: 4 	 ground: 5 
 	 T-F: False 	clever_score: 0.0056275948310269405
ID: 9 
 	 pred: 7 	 ground: 9 
 	 T-F: False 	clever_score: 0.0027546073185561203


In [26]:
R_L1 = 40
R_L2 = 2
R_LI = 0.1

def _create_krclassifier_FMNIST():
    num_classes = 10
    input_shape = (28,28,1)
    
    model = Sequential()
    # Block 1
    model.add(Conv2D(64, (3, 3), padding='same', name = 'block1_conv1', 
                     input_shape=input_shape))
    # model.add(BatchNormalization(name = 'block1_bn1'))
    model.add(Activation('relu', name = 'block1_act1'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='block1_pool1'))

    model.add(Conv2D(128, (3, 3), padding='same', name = 'block1_conv2'))
    # model.add(BatchNormalization(name = 'block1_bn2'))
    model.add(Activation('relu', name = 'block1_act2'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='block1_pool2'))

    model.add(Conv2D(256, (3, 3), padding='same', name = 'block1_conv3'))
    # model.add(BatchNormalization(name = 'block1_bn3'))
    model.add(Activation('relu', name = 'block1_act3'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='block1_pool3'))
    # top
    model.add(Flatten(name='flatten'))
    model.add(Dense(1024, activation='relu', name='fc1'))
    model.add(Dropout(0.5, name='drop1'))
    model.add(Dense(32, activation='relu', name='fc2'))
    model.add(Dropout(0.5, name='drop2'))
    # model.add(Dense(64, activation='relu', name='fc3'))
    # model.add(Dropout(0.5, name='drop3'))
    model.add(Dense(num_classes, activation='softmax', name='predictions'))

    
    # model = load_model("./fmnist_CNN.h5")
    opt = tf.keras.optimizers.Adam(learning_rate=0.00005)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])

    # Get the classifier
    krc = KerasClassifier(model=model, clip_values=(0, 1), use_logits=False)

    return krc

logger = logging.getLogger(__name__)

In [34]:
# Get Fashion-MNIST
batch_size, nb_train, nb_test = 100, 1000, 100
(x_train, y_train), (x_test, y_test), _, _ = load_mnist()
# x_train, y_train = x_train[:nb_train], y_train[:nb_train]
# x_test, y_test = x_test[:nb_test], y_test[:nb_test]

# Get the classifier
krc = _create_krclassifier_FMNIST()
krc.fit(x_train, y_train, batch_size=256, nb_epochs=5, 
        validation_data=(x_test[0:5000], y_test[0:5000]), verbose=1)

# checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath="./fmnist_CNN.h5", 
#                                                     monitor='val_accuracy', 
#                                                     verbose=1, 
#                                                     save_best_only=True, mode = 'max')
# callbacks_list = [checkpoint]
# krc.fit(x_train, y_train, batch_size=256, epochs=20, shuffle=True,
#           validation_data=(x_test[0:5000], y_test[0:5000]), callbacks=callbacks_list)

Train on 60000 samples, validate on 5000 samples
Epoch 1/5
60000/60000 [==============================] - 4s 60us/sample - loss: 1.4433 - accuracy: 0.5179 - val_loss: 0.4452 - val_accuracy: 0.8996
Epoch 2/5
60000/60000 [==============================] - 3s 47us/sample - loss: 0.5639 - accuracy: 0.8274 - val_loss: 0.2289 - val_accuracy: 0.9396
Epoch 3/5
60000/60000 [==============================] - 2s 39us/sample - loss: 0.3901 - accuracy: 0.8838 - val_loss: 0.1706 - val_accuracy: 0.9506
Epoch 4/5
60000/60000 [==============================] - 2s 39us/sample - loss: 0.3086 - accuracy: 0.9107 - val_loss: 0.1316 - val_accuracy: 0.9634
Epoch 5/5
60000/60000 [==============================] - 3s 46us/sample - loss: 0.2629 - accuracy: 0.9233 - val_loss: 0.1100 - val_accuracy: 0.9680


In [45]:
nb_test = 2000
count1 = 0
count2 = 0
for index in range(0, nb_test):
    pred = krc.predict(np.reshape(x_test[index], [-1,28,28,1]))
    if np.argmax(pred)!=np.argmax(y_test[index]):
        res_tmp = clever_u(krc, x_test[index], 10, 10, R_LI, norm=np.inf, pool_factor=3, verbose=False)
        tmp1 = tmp1+res_tmp
        print('ID:', index, '\n', '\t',
              'pred:', np.argmax(pred), '\t',
              'ground:', np.argmax(y_test[index]), '\n', '\t',
              'T-F:', np.argmax(pred)==np.argmax(y_test[index]), '\t'
              'clever_score:', res_tmp)
        count2 = count2+1
        if res_tmp<0.09:
            count1 = count1+1

ID: 151 
 	 pred: 8 	 ground: 9 
 	 T-F: False 	clever_score: 0.0008706783480636455
ID: 241 
 	 pred: 8 	 ground: 9 
 	 T-F: False 	clever_score: 0.000369658655701675
ID: 247 
 	 pred: 6 	 ground: 4 
 	 T-F: False 	clever_score: 0.0018672173484374735
ID: 259 
 	 pred: 0 	 ground: 6 
 	 T-F: False 	clever_score: 0.003911552687412376
ID: 290 
 	 pred: 4 	 ground: 8 
 	 T-F: False 	clever_score: 0.003479892490973268
ID: 320 
 	 pred: 7 	 ground: 9 
 	 T-F: False 	clever_score: 0.0004299585554347991
ID: 321 
 	 pred: 7 	 ground: 2 
 	 T-F: False 	clever_score: 0.028364724616386883
ID: 445 
 	 pred: 0 	 ground: 6 
 	 T-F: False 	clever_score: 0.08024972603931253
ID: 571 
 	 pred: 9 	 ground: 4 
 	 T-F: False 	clever_score: 5.6730817773811945e-05
ID: 578 
 	 pred: 8 	 ground: 3 
 	 T-F: False 	clever_score: 0.017135289747440007
ID: 582 
 	 pred: 2 	 ground: 8 
 	 T-F: False 	clever_score: 0.04801324618676696
ID: 591 
 	 pred: 2 	 ground: 8 
 	 T-F: False 	clever_score: 0.005298061243270832
I

In [46]:
# count1:78/100
# count1: 22/23(1000)
# count1: 57/62(2000)
print(count1, count2)

57 62


In [17]:
print(res0, res1, res2,res_tmp)

6.295663504437266 1.2352943377988042 0.0007702954387103044 0.0005591019224632063


In [32]:
tmp1 = 0
for x_tmp in x_test:
    res_tmp = clever_t(krc, x_tmp, 3, 1, 1, R_LI, norm=np.inf, pool_factor=3)
    tmp1 = tmp1+res_tmp
    print(res_tmp)
print(tmp1/10)

0.1
0.0465774285867665
0.1
0.08784920009755488
0.1
0.1
0.1
0.03720811096017883
0.05661454428495274
0.06039763977452185
0.07886469237039748


In [23]:
print(x_tmp.shape)

(28, 28, 1)


In [47]:
y_test[1]

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [40]:
x_test[9].shape

(28, 28, 1)